# MHKiT WPTO Hindcast Example

This example loads data from the [WPTO hindcast data set](https://registry.opendata.aws/wpto-pds-us-wave/), hosted on AWS.  

## Dataset Description

The high-spatial-resolution dataset (hereafter the *'spatial dataset'*) currently spans the U.S. Exclusive Economic Zone (EEZ) along the West coast and around Hawaii with an unstructured grid that has ~200 m resolution in shallow water. The time step resolution for this spatial data is 3-hours and spans 32 years, 01/01/1979  - 12/31/2010. 

        
- Dataset variables included are indexed by **'coordinates'** (latitude and longitude), and a **'time_index'**

### Included Spatial Variables: 

| Variable Name | Units |
| :------------ | :---: |
| energy_period | seconds |
| maximum_energy_period | degrees_true |
| mean_absolute_period | seconds |
| mean_zero-crossing_period | seconds |
| omni-directional_wave_power | Watts |
| peak_period | seconds |
| significant_wave_height | meters |
| water_depth | meters |
| spectral_width | none |
| directionality_coefficient | none |


### Setting up Access to WPTO Hindcast Data
To access the WPTO hindcast data, you will need to configure h5pyd for data access on HSDS. 
        To get your own API key, visit https://developer.nrel.gov/signup/. 

To configure h5phd type:

    hsconfigure
        
and enter at the prompt:

    hs_endpoint = https://developer.nrel.gov/api/hsds
    hs_username = None
    hs_password = None
    hs_api_key = {your key}

 
## Using the WPTO Hindcast MHKiT Function

In this section we will walk through an example to request hindcast data at a point using MHKiT. To start we will import the MHKiT wave module.

In [11]:
from mhkit import wave

### 1. Download WPTO Hindcast Data from a Single Location

In this example we will request significant wave height for 1995 at the point (44.624076,-124.280097, near the PacWave site). We will speficy each of these as a variable and pass them to the wave IO hindacast function `request_wpto_dataset`. The data type will be a high-spatial-resolution dataset,  year(s) will be 1995,  latitude/longitude pairs will be for a point near PacWave, and the parameter will be significant wave height.

In [12]:
data_type = 'spatial' 
years = [1995]
lat_lon = (44.624076,-124.280097) 
parameter = 'significant_wave_height' 

Hs, metadata= wave.io.hindcast.request_wpto_dataset(data_type,parameter,lat_lon,years)

#### View the metadata
We can check the metadata of our request to find information such as water depth, distance to shore, timezone (offset from UTC), and jurisdiction. 

In [13]:
metadata

,water_depth,latitude,longitude,distance_to_shore,timezone,jurisdiction
0,77.429497,44.624298,-124.278999,15622.175781,-8,Federal


#### Hindacast Data Returned 

We can also take a look at the first 5 rows of the returned data by call the `head()` method on the returned data. By calling this below we can see that the returned data has a time index and a single column of significant wave height. The siginifcant wave height column has a 0 integer on the end of it reffering to the index in the metadata. In out next example the importance of this integer will become more clear as we begin to request multiple locations.

In [4]:
Hs.head()

,significant_wave_height_0
time_index,
1995-01-01 00:00:00+00:00,2.35354
1995-01-01 03:00:00+00:00,2.39468
1995-01-01 06:00:00+00:00,2.45756
1995-01-01 09:00:00+00:00,2.55913
1995-01-01 12:00:00+00:00,2.66992


## 2. Download WPTO Hindcast Data from Multiple Locations

Multiple locations can be requested by passing a list or tuple of lat/lon pairs. This time we will request the energy period and two points. The data type and years requested remanin the same as before. By looking at the `head` of the returned DataFrame we can see that this time two energy periods are returned. The integers on each of the energy periods refer to the index in the metadata. This allows the user to connect the quantities of interest and the metadata fro each point.

In [15]:
parameter = 'energy_period'
lat_lon = ((44.624076,-124.280097),
           (43.489171,-125.152137)) 

Te, metadata = wave.io.hindcast.request_wpto_dataset(data_type, parameter, lat_lon, years)

# View Te from two locations
Te.head()

,energy_period_0,energy_period_1
time_index,,
1995-01-01 00:00:00+00:00,10.3433,10.1760
1995-01-01 03:00:00+00:00,10.2281,10.2641
1995-01-01 06:00:00+00:00,10.0164,10.1792
1995-01-01 09:00:00+00:00,10.0484,10.2410
1995-01-01 12:00:00+00:00,10.6579,10.5877


In [16]:
metadata

,water_depth,latitude,longitude,distance_to_shore,timezone,jurisdiction
0,77.429497,44.624298,-124.278999,15622.175781,-8,Federal
1,1337.407959,43.496300,-125.152000,64236.390625,-8,Federal


## 3. Download WPTO Hindcast Data from Multiple Years

Multiple years can be requested simply by adding years to the passed list. Recall the hindcast data spans  01/01/1979  - 12/31/2010. A demonstration of requesting multiple years of spatial data for a single point is shown below. By looking at the the returned data we can see that the data is returned in the order of the years passed to the function. 

In [19]:
years = [1995, 1996] 
parameter = 'omni-directional_wave_power'
lat_lon = (44.624076,-124.280097) 

J, metadata= wave.io.hindcast.request_wpto_dataset(data_type,parameter,lat_lon,years) 

J

,omni-directional_wave_power_0
time_index,
1995-01-01 00:00:00+00:00,30134.0
1995-01-01 03:00:00+00:00,30805.0
1995-01-01 06:00:00+00:00,31709.0
1995-01-01 09:00:00+00:00,34476.0
1995-01-01 12:00:00+00:00,39892.0
...,...
1996-12-31 09:00:00+00:00,154849.0
1996-12-31 12:00:00+00:00,120543.0
1996-12-31 15:00:00+00:00,108288.0


In [18]:
metadata

,water_depth,latitude,longitude,distance_to_shore,timezone,jurisdiction
0,77.429497,44.624298,-124.278999,15622.175781,-8,Federal
